#!pip install -U pandas-profiling


In [1]:
!pip install -U pandas-profiling

    100% |████████████████████████████████| 133kB 2.6MB/s ta 0:00:01


  Stored in directory: /Users/karthikmahendra/Library/Caches/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
Successfully built pandas-profiling
  Found existing installation: pandas-profiling 2.1.2
    Uninstalling pandas-profiling-2.1.2:
      Successfully uninstalled pandas-profiling-2.1.2


In [26]:
import numpy as np 
import pandas as pd
import pandas_profiling 
import datetime as dt
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 
import category_encoders as ce
from sklearn.model_selection import train_test_split


In [27]:
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/tanzania/'

train_features = pd.read_csv(WEB + 'train_features.csv')
train_labels = pd.read_csv(WEB + 'train_labels.csv')
test_features = pd.read_csv(WEB + 'test_features.csv')
sample_submission = pd.read_csv(WEB + 'sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

PRE-PROCESS 

In [28]:
#Explore data

profile = train_features.profile_report()
profile.to_file(output_file = 'output.html')

In [29]:
train_labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [30]:
# from output.html we have 10 numerical and 26 categorical features. 2 boolean and 2 rejected. 

bool_features = ['public_meeting','permit']
rejected_features = ['quantity_group','recorded_by']
categorical_features = train_features.select_dtypes(exclude=[np.number]).columns.to_list()
numerical_features = train_features.select_dtypes(include=[np.number]).columns.to_list()

In [31]:
#Remove boolean values from categorical_features
categorical_features = list(set(categorical_features) - set(bool_features))

In [32]:
print(categorical_features)

['wpt_name', 'region', 'scheme_name', 'extraction_type', 'ward', 'waterpoint_type_group', 'recorded_by', 'extraction_type_class', 'scheme_management', 'quality_group', 'waterpoint_type', 'water_quality', 'source', 'quantity', 'management_group', 'source_class', 'management', 'extraction_type_group', 'date_recorded', 'installer', 'subvillage', 'payment_type', 'source_type', 'basin', 'funder', 'lga', 'quantity_group', 'payment']


In [33]:
print(numerical_features)

['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population', 'construction_year']


In [34]:
# find if there are any NANs in categorical_features 

train_features[categorical_features].isnull().values.any()

True

In [35]:
# find if there are any NANs in numerical_features 

train_features[numerical_features].isnull().values.any()

False

In [36]:
#Since null values are only in categorical features, 
#I will find the top values in each and replace NANs with that value
def clean_data(lis):
    for element in lis:
        top = train_features[element].describe().to_list()[2]
        train_features[element]=train_features[element].fillna(top)
    

In [37]:
clean_data(categorical_features)

In [38]:
train_features[categorical_features].isnull().values.any()

False

In [39]:
train_features.isnull().values.any()

True

SPLIT

In [40]:
X = train_features
y = train_labels['status_group']
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, stratify=y)

In [41]:
type(X_train),y_train.shape,X_val.shape,y_val.shape


(pandas.core.frame.DataFrame, (44550,), (14850, 40), (14850,))

LOGISTIC REGRESSION

In [42]:
# Select Categorical features with <19 values to train dataset
cat_5 = []
for ele in categorical_features:
    if len(train_features[ele].unique().tolist())<19:
        #print(f'{ele}: {len(train_features[ele].unique().tolist())}')
        cat_5.append(ele)
print (cat_5)
    

['extraction_type', 'waterpoint_type_group', 'recorded_by', 'extraction_type_class', 'scheme_management', 'quality_group', 'waterpoint_type', 'water_quality', 'source', 'quantity', 'management_group', 'source_class', 'management', 'extraction_type_group', 'payment_type', 'source_type', 'basin', 'quantity_group', 'payment']


In [43]:
features = cat_5 + numerical_features

In [44]:
X_train_subset = X_train[features]
X_val_subset = X_val[features]
X_test_subset = test_features[features]

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset[cat_5])
X_val_encoded = encoder.transform(X_val_subset[cat_5])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)



/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


In [45]:
model = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=10000)
model.fit(X_train_scaled, y_train)
print('Validation Accuracy', model.score(X_val_scaled, y_val))

Validation Accuracy 0.7288215488215488


PREDICTION

In [46]:
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset[cat_5])
X_test_encoded = encoder.transform(X_test_subset[cat_5])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  import sys


In [47]:
y_pred = model.predict(X_test_encoded)

In [48]:
y_pred

array(['non functional', 'functional', 'non functional', ...,
       'functional', 'functional', 'non functional'], dtype=object)

In [49]:
len (y_pred)

14358

In [ ]:
# estimator is your scikit-learn estimator, which you've fit on X_train

# X_test is your pandas dataframe or numpy array, 
# with the same number of rows, in the same order, as test_features.csv, 
# and the same number of columns, in the same order, as X_train



# Makes a dataframe with two columns, id and status_group, 
# and writes to a csv file, without the index
sample_submission.shape[0]

 
# submission = sample_submission.copy()
# submission['status_group'] = y_pred
# submission.to_csv('sample_submission_0.csv', index=False)

In [50]:
# Write submission csv file
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-01.csv', index=False)

Submission to Kaggle 